# Deep learning to predict structured information from Bogotá Complaints Data

In [24]:
#read data choose between 2017 and 2016
import numpy as np
import pandas as pd
df = pd.read_csv('data/SDQS 2017-enejunV2.csv')
#df = pd.read_excel('data/SDQS 2016-enedic.xlsx')
df= df.sample(frac=1)

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [55]:
#import other libraries
from datetime import datetime
import json
from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation,GRU,Convolution1D,Flatten, GlobalMaxPooling1D
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

In [32]:
#cut low frequencies

from nltk.stem import  SnowballStemmer

featureToPredict = 'Entidad'

balanced_texts = []
balanced_labels = []

balanced_texts = list(df['Asunto'])
    
#replace low freq# run if needed
count_table = pd.Series(df[featureToPredict]).value_counts()
low_freq_list = list(count_table[count_table<2500].index)
Y = pd.Series(df[featureToPredict]).apply(lambda x: 'OTHER' if x in low_freq_list else x)
balanced_labels = pd.get_dummies(Y)

(a,n) = balanced_labels.shape
n

13

In [33]:
import re
from collections import Counter
from nltk.stem import  SnowballStemmer
stemmer = SnowballStemmer('spanish')

#build a flat list
#flat_list = [item for sublist in balanced_texts for item in sublist]
#counter = Counter(flat_list)

#flat word list lower
flat_list = []
for complaint in balanced_texts:
    for word in complaint.lower().split():
        flat_list.append(word)

#remove punctuactions and numbers
punctuation = re.compile(r'[-¿><`…/&#.?–“!,":;()|0-9]')
flat_list_cleaned = [punctuation.sub("", word) for word in flat_list]



#stemming
flat_list_cleaned_stemed = [stemmer.stem(i) for i in flat_list_cleaned]
counter = Counter(flat_list_cleaned_stemed)       
print(len(counter))

80696


### word embedding with Keras

In [53]:
#parameters
nWordsEmbedding = 10000
maxLen = 300
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250

In [52]:
#word embedding with Keras
tokenizer = Tokenizer(num_words=nWordsEmbedding)
tokenizer.fit_on_texts(balanced_texts)
sequences = tokenizer.texts_to_sequences(balanced_texts)
data = pad_sequences(sequences, maxlen=maxLen)

In [56]:
model = Sequential()
model.add(Embedding(nWordsEmbedding,embedding_dims, input_length=maxLen))
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Dense(n,activation='softmax'))

#optimizer sgd is slower 30'
#optimizaer adam is faster
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [57]:
maxLen

300

In [58]:
model.fit(np.array(data), np.array(balanced_labels), validation_split=0.3, epochs=2)
#model.summary()

#verify accuracy by class!!

Train on 102186 samples, validate on 43794 samples
Epoch 1/2
102186/102186 [==============================] - 2114s - loss: 0.7694 - acc: 0.7517 - val_loss: 0.5960 - val_acc: 0.8024
Epoch 2/2
102186/102186 [==============================] - 995s - loss: 0.5048 - acc: 0.8306 - val_loss: 0.6305 - val_acc: 0.7946


In [59]:
#accuracy by Entidad
predictions = model.predict(np.array(data))
accurVector = np.argmax(predictions,1)==np.argmax(np.array(balanced_labels),1)
s = pd.Series(accurVector, index= np.argmax(np.array(balanced_labels),1))

print(s.groupby(by=np.argmax(np.array(balanced_labels),1)).mean())
print(s.groupby(by=np.argmax(np.array(balanced_labels),1)).count())

0     0.847077
1     0.880606
2     0.962260
3     0.851986
4     0.873507
5     0.691041
6     0.886216
7     0.658541
8     0.850967
9     0.935207
10    0.470943
11    0.892053
12    0.989475
dtype: float64
0     36162
1      6935
2     32406
3      7479
4      6364
5      4007
6      4614
7     23341
8      4858
9      4661
10     2822
11     3020
12     9311
dtype: int64


In [ ]:
#best model
model = Sequential()
model.add(Embedding(nWordsEmbedding,embedding_dims, input_length=maxLen))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Dense(n,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

##7946 ACCURACY

# Pieces of code to use later

In [62]:
for i in balanced_labels.columns:
    print(i)


OTHER
SECRETARIA DE AMBIENTE
SECRETARIA DE GOBIERNO
SECRETARIA DE INTEGRACION SOCIAL
SECRETARIA DE SALUD
SECRETARIA DE SEGURIDAD (NUEVA)
SECRETARIA DEL HABITAT
SECRETARIA GENERAL
SECRETARIA MOVILIDAD
SUBRED SUR (NUEVA)
TRANSMILENIO
UAESP - UNIDAD DE SERVICIOS PUBLICOS
UNIVERSIDAD DISTRITAL


In [70]:
df[df['Entidad']=="TRANSMILENIO"]['Asunto'][129603]

'PESIMO SERVICIO EN PORTAL 80, NO HAY BUENA FRECUENCIA DE BUSES, ESTO HACE QUE LA PLATAFORMA SE CONGESTIONE Y NO SEA POSIBLE TOMAR EL BUS QUE SE NECESITA, ESTAMOS OBLIGADOS A TOMAR ESTE SERVICIO Y NO HAY SOLUCIONES ARA LO USUARIOS, LO UNICO IMPORTANTE PARA EL SISTEMA ES LUCRARSE PERO NO DA COMIDAD A LOS USUARIOS.'

In [132]:
accurVector = np.argmax(predictions,1)==np.argmax(np.array(balanced_labels),1)
s = pd.Series(accurVector, index= np.argmax(np.array(balanced_labels),1))

print(s.groupby(by=np.argmax(np.array(balanced_labels),1)).mean())
print(s.groupby(by=np.argmax(np.array(balanced_labels),1)).count())



0     0.823093
1     0.805308
2     0.793935
3     0.922460
4     0.600068
5     0.881409
6     0.840355
7     0.836005
8     0.805861
9     0.447235
10    0.921076
11    0.685604
dtype: float64
0      8756
1      5727
2      2572
3     11852
4     23574
5     35222
6     12058
7      7799
8     11260
9      5335
10    17143
11     4682
dtype: int64


145980

In [9]:
model = Sequential()
model.add(Embedding(5000, 128, input_length=300))
# the model will take as input an integer matrix of size (batch, input_length).
# the largest integer (i.e. word index) in the input should be no larger than 999 (vocabulary size).
# now model.output_shape == (None, 10, 64), where None is the batch dimension.
model.add(LSTM(128)) #, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(12, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

#input_array = np.random.randint(5000, size=(32, 300))
#output_array = model.predict(input_array)
#print(output_array.shape)

In [6]:
balanced_texts

['CIUDADANO SE COMUNICA EL DIA DE HOY 16 DE AGOSTO DE 2016, MENCIONA QUE ALREDEDOR DE LAS 9:45AM SE ACERCO AL PUNTO DE ATENCION CAMI PATIO BONITO, QUE PERTENCE AL HOSPITAL DE KENNEDY; CON EL PACIENTE JOSE EMILIO SIERRA, IDENFICADO CON NUMERO DE DOCUMENTIO 6027144. INGRESO CON UNA HERIDA EN SU OJO DE DERECHO, ACOMPAÑADO POR POLICIA METROPOLITANA DE BOGOTA. DONDE EL AUXILIAR DE ENFERMERIA JORGE MOLLEDO LE NEGO EL SERVICIO Y FUE DEMASIADO GROSERO AL MOMENTO DE PRESTARLE SU ATENCION. EL PACIENTE SIENDO HABITANTE DE LA CALLE INGRESO AL SERVICIO DE URGENCIAS. DONDE INDICA EL CIUDADANO QUE LE INDICO QUE PARA INGRESABAN AL PUNTO,  SI ERA HABITANTE DE LA CALLE, EL NO MERECIA SU ATENCION Y QUE PORQUE NO SE IBAN PARA OTRO PUNTO DE ATENCION. ACUDIO AL CAMI, PUNTO DE ATENCION MAS CERCANO PARA QUE LOGRARAN ATENDER; YA QUE LA POLICIA METROPOLITANA LOGRO LLEVARLO PRONTAMENTE. CIUDADANO MANIFIESTA UN LLAMADO DE ATENCION PARA ESTA PERSONA QUE LO ATENCION Y QUE POR FAVOR SE REVISEN LOS HECHOS; YA QUE POR

In [26]:
pd.factorize(Y)[0]

array([0, 1, 2, ..., 4, 5, 2])

In [21]:
### use other classifier get the sparsed matrix
zeros = 0
dataMatrix = np.zeros((balanced_labels.shape[0],5000))
for index, idArray in enumerate(sequences):
    for idCol in idArray:
        dataMatrix[index,idCol] = 1


In [27]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test = train_test_split(dataMatrix,pd.factorize(Y)[0],random_state=0)

In [ ]:
#### xgboost classification ## skip to use KERAS

#classification
import xgboost as xgb
xg_train = xgb.DMatrix(X_train, label=y_train)
xg_test = xgb.DMatrix(X_test, label=y_test)

param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['num_class'] = len(pd.factorize(Y)[0])

watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 5
bst = xgb.train(param, xg_train, num_round, watchlist)
# get prediction
pred = bst.predict(xg_test)
error_rate = np.sum(pred != y_test) / y_test.shape[0]
print('Test error using softmax = {}'.format(error_rate))



//anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [215]:
model = Sequential()
model.add(Embedding(input_dim=10000, 
          output_dim=12, 
          embeddings_initializer='uniform', 
          embeddings_regularizer=None, 
          activity_regularizer=None, 
          embeddings_constraint=None, 
          mask_zero=False, 
          input_length=150))

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

input_array = data[0:32,:]

model.add(Dense(12))
#model.add(Activation('softmax'))

#model.add(Activation('relu'))
#model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
#model.add(Dense(120, activation='softmax'))
#model.add(Dense(12, activation='softmax'))

#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

output_array = model.predict(input_array)
assert output_array.shape == (32, 150, 12)

In [216]:
model.fit(data, np.array(balanced_labels), validation_split=0.5, epochs=1)

ValueError: Error when checking target: expected dense_38 to have 3 dimensions, but got array with shape (145980, 12)

In [211]:
np.array(balanced_labels).shape

(145980, 12)

In [2]:
#tokenize documents
from collections import namedtuple
from nltk.stem import  SnowballStemmer
from nltk.tokenize import word_tokenize

    
stemmer = SnowballStemmer('spanish')
docs = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
for i, text in enumerate(df['Asunto']):
    #words = text.lower().split()
    words = word_tokenize(text.lower())
    tags = [i]
    docs.append(analyzedDocument(words, tags))
    
#docs_w2v = []

#for i, text in enumerate(df['Asunto']):
#    words = text.lower().split()
#    words_stemm = [stemmer.stem(i) for i in words]
#    docs_w2v.append(words_stemm)

In [128]:
#apply doc2vec train the model 
#skip in pretrained is used

from gensim.models import doc2vec,word2vec
model = word2vec.Word2Vec(docs_w2v, size = 100, window = 300, min_count = 10, workers = 4)

NameError: name 'docs_w2v' is not defined

In [3]:
# load pre-trained model 
## word embedding ##
#from gensim.models import Word2Vec
import gensim.models.keyedvectors as word2vec

# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
model = word2vec.KeyedVectors.load_word2vec_format(
    '~/Dropbox/Maldonado/SBW-vectors-300-min5.bin', binary=True)


Using TensorFlow backend.


In [4]:
#COMBINE THE EMBEDDINGS OPTION 1#
#create a vector of average the words.
average_vect =[]
for sen in docs:    
    ave = np.zeros(300)
    for word_ in sen[0]:
        try:
            ave = ave + model[word_]/len(sen)
        except:
            #print('word: ' + str(word_) + ' not found')
            pass
    average_vect.append(ave)        

In [ ]:
#COMBINE THE EMBEDDINGS OPTION 2-
#create cluster n=3 of each complaint. concatenate the centers.
centers_c_vect =[]



In [120]:
#for xgboost
#Y = pd.factorize(s)[0]

#for keras
#Y = pd.get_dummies(df['Sector'])

In [12]:
#replace low freq# run if needed
Y= df['Sector']
s= pd.Series(Y)
count_table = s.value_counts()
low_freq_list = list(count_table[count_table<2500].index)
Y = Y.apply(lambda x: 'OTHER' if x in low_freq_list else x)

In [24]:
X = np.matrix(average_vect)

#factorize for Keras
Y = pd.get_dummies(Y)

#xgboost
#Y = pd.factorize(Y)[0]

In [25]:
Y

,0,1,2,3,4,5,6,7,8,9,10,11
0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0
5,0,0,1,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,0,0
7,0,1,0,0,0,0,0,0,0,0,0,0
8,0,0,0,1,0,0,0,0,0,0,0,0
9,0,0,0,0,1,0,0,0,0,0,0,0


In [26]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test = train_test_split(X,Y,random_state=0)


In [27]:
#### xgboost classification ## skip to use KERAS


#classification
import xgboost as xgb
xg_train = xgb.DMatrix(X_train, label=y_train)
xg_test = xgb.DMatrix(X_test, label=y_test)

param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['num_class'] = max(Y)+1

watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 5
bst = xgb.train(param, xg_train, num_round, watchlist)
# get prediction
pred = bst.predict(xg_test)
error_rate = np.sum(pred != y_test) / y_test.shape[0]
print('Test error using softmax = {}'.format(error_rate))



ValueError: DataFrame for label cannot have multiple columns

In [57]:
## keras deep learning ##
from keras.models import Sequential
model = Sequential()

In [58]:
from keras.layers import Dense, Activation

model.add(Dense(units=240, input_dim=300))
model.add(Activation('relu'))
model.add(Dense(units=12))
model.add(Activation('softmax'))

In [59]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [54]:
# x_train and y_train are Numpy arrays --just like in the Scikit-Learn API.
model.fit(np.array(X_train), np.array(y_train), epochs=10, batch_size=32)
#model.train_on_batch(x_batch, y_batch)


#Evaluate your performance in one line:
loss_and_metrics = model.evaluate(np.array(X_test), np.array(y_test), batch_size=128)

#Or generate predictions on new data:
classes = model.predict(np.array(X_test), batch_size=128)

print(sum(np.argmax(classes, axis=1)==np.argmax(np.array(y_test), axis=1))/len(np.argmax(classes, axis=1)))

Epoch 1/10
109485/109485 [==============================] - 9s - loss: 0.6576 - acc: 0.7862     
Epoch 2/10
109485/109485 [==============================] - 8s - loss: 0.6540 - acc: 0.7878     
Epoch 3/10
109485/109485 [==============================] - 9s - loss: 0.6500 - acc: 0.7890     
Epoch 4/10
109485/109485 [==============================] - 8s - loss: 0.6473 - acc: 0.7894     
Epoch 5/10
109485/109485 [==============================] - 8s - loss: 0.6432 - acc: 0.7902     
Epoch 6/10
109485/109485 [==============================] - 8s - loss: 0.6432 - acc: 0.7913     
Epoch 7/10
109485/109485 [==============================] - 8s - loss: 0.6391 - acc: 0.7913     
Epoch 8/10
109485/109485 [==============================] - 8s - loss: 0.6350 - acc: 0.7933     
Epoch 9/10
109485/109485 [==============================] - 9s - loss: 0.6325 - acc: 0.7936     
Epoch 10/10
34688/36495 [===========================>..] - ETA: 0s0.762597616112


In [46]:
sum(np.argmax(classes, axis=1)==np.argmax(np.array(y_test), axis=1))/len(np.argmax(classes, axis=1))

0.76100835730922045

In [2]:
import tensorflow as tf
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

b'Hello, TensorFlow!'


In [68]:
### keras with iris###
type(np.array(y_test))

pandas.core.frame.DataFrame

In [32]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load dataset
dataframe = pandas.read_csv("iris.csv") #, header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]


In [36]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=4, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Baseline: 97.33% (4.42%)


In [86]:
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=4, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model


In [20]:
### wordtovec with subtema.
### NOT WORKING ###

from collections import namedtuple
from nltk.stem import  SnowballStemmer
stemmer = SnowballStemmer('spanish')

#word to vec with subtema
Subtema = np.unique(df['Subtema'])

#replace low freq# run if needed
#df_temp = pd.DataFrame(Subtema)
#df_temp2 = df_temp.apply(lambda x: x.mask(x.map(x.value_counts())<15, 'OTROS') if x.name!='0' else x)
#Y = df_temp2

docs_w2v=[]

for i, text in enumerate(Subtema):
    words = text.lower().split()
    words_stemm = [stemmer.stem(j) for j in words]
    docs_w2v.append(words_stemm)

In [35]:
from gensim.models import doc2vec,word2vec
model = word2vec.Word2Vec(docs_w2v, size = 200, window = 300, min_count = 1, workers = 4)


In [36]:
## biuld the average by row
average_vect =[]
for sen in docs_w2v:
    
    ave = np.zeros(200)
    for word in sen:

        ave = ave + model[word]/len(sen)        
    average_vect.append(ave)

In [38]:
from sklearn.cluster import SpectralClustering
mat = np.matrix(average_vect)
labels = SpectralClustering(100).fit_predict(mat)

In [32]:
import numpy as np
from sklearn.cluster import KMeans

### For the purposes of this example, we store feature data from our
### dataframe `df`, in the `f1` and `f2` arrays. We combine this into
### a feature matrix `X` before entering it into the algorithm.
#f1 = df['Distance_Feature'].values
#f2 = df['Speeding_Feature'].values

X=np.matrix(average_vect)
kmeans = KMeans(n_clusters=100).fit(X)

In [39]:
data = {'subtema': Subtema, 'cluster':labels}
pd.DataFrame(data).to_csv('subtema.csv')

In [39]:
balanced_labels.shape

(145980, 13)